In [ ]:
import csv

import cv2
import numpy as np
import matplotlib.pyplot as plt

In [9]:
lines = []
with open('./data/track_1_straight/driving_log.csv') as csvfile:
    reader = csv.reader(csvfile)
    for line in reader:
        lines.append(line)

images = []
measurements = []
for line in lines:
    current_path = line[0]
    
    image = plt.imread(current_path)
    images.append(image)
    
    measurement = float(line[3])
    measurements.append(measurement)
    
X_train = np.array(images)
y_train = np.array(measurements)

In [10]:
from keras.models import Sequential
from keras.layers import Flatten, Dense

Using TensorFlow backend.


In [11]:
model = Sequential()
model.add(Flatten(input_shape=(160,320,3)))
model.add(Dense(1))

model.compile(loss='mse', optimizer='adam')
model.fit(X_train, y_train, validation_split=0.2, shuffle=True, epochs=7)

model.save('model.h5')

Train on 2260 samples, validate on 565 samples
Epoch 1/7
2260/2260 [==============================] - 57s 25ms/step - loss: 16373640.1826 - val_loss: 13456.2885
Epoch 2/7
2260/2260 [==============================] - 2s 771us/step - loss: 23238.9943 - val_loss: 5708.1034
Epoch 3/7
2260/2260 [==============================] - 2s 769us/step - loss: 6437.4921 - val_loss: 5025.5605 ETA: 0s - loss: 6722. - ETA: 0s - loss: 659
Epoch 4/7
2260/2260 [==============================] - 2s 768us/step - loss: 4585.7945 - val_loss: 5471.7229
Epoch 5/7
2260/2260 [==============================] - 2s 766us/step - loss: 4381.3855 - val_loss: 3302.7594
Epoch 6/7
2260/2260 [==============================] - 2s 750us/step - loss: 3477.2603 - val_loss: 2797.0399
Epoch 7/7
2260/2260 [==============================] - 2s 746us/step - loss: 3590.5719 - val_loss: 6742.7475
